# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [1]:
import pandas as pd

url_file1 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv"
url_file2 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv"
url_file3 = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv"

data_file1 = pd.read_csv(url_file1)
data_file2 = pd.read_csv(url_file2)
data_file3 = pd.read_csv(url_file3)

# Concatenate the three DataFrames into a single DataFrame
combined_data = pd.concat([data_file1, data_file2, data_file3], ignore_index=True)

In [2]:
# Cleaning the data

# Combine the values from 'ST' and 'State' columns into a new 'state' column
combined_data['state'] = combined_data['ST'].fillna(combined_data['State'])

# Combine the values from 'GENDER' and 'Gender' columns into a new 'gender' column
combined_data['gender'] = combined_data['GENDER'].fillna(combined_data['Gender'])

# Place the 'state' column as the second column in the DataFrame
combined_data = combined_data[['Customer', 'state'] + [col for col in combined_data if col != 'Customer' and col != 'state']]

# Place the 'gender' column as the third column in the DataFrame
combined_data = combined_data[['Customer', 'state', 'gender'] + [col for col in combined_data if col != 'Customer' and col != 'state' and col != 'gender']]

# Drop the 'ST' and 'State' columns from the DataFrame
combined_data.drop(columns=['ST', 'State'], inplace=True)

# Drop the 'GENDER' and 'Gender' columns from the DataFrame
combined_data.drop(columns=['GENDER', 'Gender'], inplace=True)

# From this point I just used my previous cleaning process from Lab 2.

# Modifies the column names.
combined_data.columns = combined_data.columns.str.lower()  # Convert column names to lowercase
combined_data.columns = combined_data.columns.str.replace(' ', '_')  # Replace white spaces with underscores
combined_data.rename(columns={'st': 'state'}, inplace=True)  # Replace 'st' with 'state'

# Clean the 'gender' column
combined_data['gender'] = combined_data['gender'].str.lower().replace({'f': 'F', 'femal': 'F', 'male': 'M', 'm': 'M'})

# Clean the 'state' column
state_mapping = {
    'AZ': 'Arizona',
    'Cali': 'California',
    'WA': 'Washington'
}
combined_data['state'] = combined_data['state'].replace(state_mapping)

# Clean the 'education' column
combined_data['education'] = combined_data['education'].replace({'Bachelors': 'Bachelor'})

# Clean the 'customer lifetime value' column
combined_data['customer_lifetime_value'] = combined_data['customer_lifetime_value'].str.rstrip('%')

# Clean the 'vehicle class' column
vehicle_mapping = {
    'Sports Car': 'Luxury',
    'Luxury SUV': 'Luxury',
    'Luxury Car': 'Luxury'
}
combined_data['vehicle_class'] = combined_data['vehicle_class'].replace(vehicle_mapping)

# Convert 'Customer Lifetime Value' column to numeric
combined_data['customer_lifetime_value'] = pd.to_numeric(combined_data['customer_lifetime_value'], errors='coerce')

# Clean and convert 'Number of Open Complaints' column to numeric
combined_data['number_of_open_complaints'] = combined_data['number_of_open_complaints'].str.split('/').str[1]
combined_data['number_of_open_complaints'] = pd.to_numeric(combined_data['number_of_open_complaints'], errors='coerce')

combined_data['number_of_open_complaints'][12] # Just to show an example. Row number 13 had a value of '1/2/0' and it turned to '2'.

# Identify columns with null values and count the null values in each column
null_counts = combined_data.isnull().sum()

# Drop rows with null values in categorical columns
categorical_columns = combined_data.select_dtypes(include=['object']).columns
combined_data = combined_data.dropna(subset=categorical_columns)

# Fill null values for numerical columns with the mean
numerical_columns = combined_data.select_dtypes(include=['float64']).columns
combined_data[numerical_columns] = combined_data[numerical_columns].fillna(combined_data[numerical_columns].mean())

# Convert numeric variables to integers
combined_data[numerical_columns] = combined_data[numerical_columns].astype(int)

# Identify duplicate rows and display the number of duplicate rows
num_duplicates = combined_data.duplicated().sum()

# Dropping duplicate rows
combined_data = combined_data.drop_duplicates()
combined_data

,customer,state,gender,education,customer_lifetime_value,income,monthly_premium_auto,number_of_open_complaints,policy_type,vehicle_class,total_claim_amount
1,QZ44356,Arizona,F,Bachelor,697953,0,94,0,Personal Auto,Four-Door Car,1131
2,AI49188,Nevada,F,Bachelor,1288743,48767,108,0,Personal Auto,Two-Door Car,566
3,WW63253,California,M,Bachelor,764586,0,106,0,Corporate Auto,SUV,529
4,GA49547,Washington,M,High School or Below,536307,36357,68,0,Personal Auto,Four-Door Car,17
5,OC83172,Oregon,F,Bachelor,825629,62902,69,0,Personal Auto,Two-Door Car,159
...,...,...,...,...,...,...,...,...,...,...,...
12069,LA72316,California,M,Bachelor,780191,71941,73,0,Personal Auto,Four-Door Car,198
12070,PK87824,California,F,College,780191,21604,79,0,Corporate Auto,Four-Door Car,379
12071,TD14365,California,M,Bachelor,780191,0,85,0,Corporate Auto,Four-Door Car,790
12072,UP19263,California,M,College,780191,21941,96,0,Personal Auto,Four-Door Car,691


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [3]:
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
marketing_data = pd.read_csv(url)

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [5]:
# Create a pivot table showing total revenue for each sales channel
sales_channel_revenue = marketing_data.pivot_table(index='sales_channel', values='total_claim_amount', aggfunc='sum').round(2)
print(sales_channel_revenue)

# In conclusion, the sales channel with the most total revenue was the Agent
# The instructions say that the values for the sales channel are: (branch, call center, web, and mail. I'm confused on why 'Agent' is in the data.

               total_claim_amount
sales_channel                    
Agent                  1810226.82
Branch                 1301204.00
Call Center             926600.82
Web                     706600.04


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [ ]:
# Your code goes here